<a href="https://colab.research.google.com/github/NadiaHolmlund/BDS_M2_NLP_Classification_App/blob/main/BDS_M2_NLP_Classification_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweet-preprocessor -q

# Installing Gensim and PyLDAvis
!pip install -qq -U gensim
!pip install -qq pyLDAvis

     |████████████████████████████████| 24.1 MB 2.3 MB/s 
     |████████████████████████████████| 1.7 MB 7.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done


In [ ]:
# explainability (why did the model say it's hate speech)
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 4.9 MB/s 
     |████████████████████████████████| 133 kB 59.1 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=3b8b86109b16ad8e7a3b0c7677ff7e155c8a0e3e10850b03a516908f88d2a4dc
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import preprocessor as prepro # twitter prepro
import tqdm #progress bar

import spacy #spacy for quick language prepro
nlp = spacy.load('en_core_web_sm') #instantiating English module

# sampling, splitting
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split


# loading ML libraries
from sklearn.pipeline import make_pipeline #pipeline creation
from sklearn.feature_extraction.text import TfidfVectorizer #transforms text to sparse matrix
from sklearn.linear_model import LogisticRegression #Logit model
from sklearn.metrics import classification_report #that's self explanatory
from sklearn.decomposition import TruncatedSVD #dimensionality reduction
from xgboost import XGBClassifier

import altair as alt #viz

#explainability
import eli5
from eli5.lime import TextExplainer

# topic modeling

from gensim.corpora.dictionary import Dictionary # Import the dictionary builder
from gensim.models import LdaMulticore # we'll use the faster multicore version of LDA

# Import pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

%matplotlib inline
pyLDAvis.enable_notebook()

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [ ]:
# prepro settings
prepro.set_options(prepro.OPT.URL, prepro.OPT.NUMBER, prepro.OPT.RESERVED, prepro.OPT.MENTION, prepro.OPT.SMILEY)

In [ ]:
import pandas as pd

In [ ]:
pol_tweets = pd.read_json('https://github.com/SDS-AAU/SDS-master/raw/master/M2/data/pol_tweets.gz') #.gz zips the json file

In [ ]:
pol_tweets.head()

,text,labels
340675,RT @GreenBeretFound Today we remember Sgt. 1st...,0
289492,"Yes, yes, yes, yes and yes. 😷 #JerseyStrong 💪🏾...",1
371088,Made new friends this afternoon delivering mas...,1
82212,RT @TXMilitary Happening TODAY: Pilots with th...,0
476047,RT @SteveScalise President Trump's legal team ...,0


In [ ]:
debate_tweets = pd.read_json('https://github.com/SDS-AAU/SDS-master/raw/master/M2/data/pres_debate_2020.gz')

In [ ]:
debate_tweets.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,nretweets,quote_url,search,near,geo,source,reply_to,translate,trans_src,trans_dest
0,1318944772183281664,1318944772183281664,2020-10-21 15:58:33,2020-10-21 15:58:33,0,,Still time to register: Students can join the ...,en,[presidentialdebate2020],[],...,2,,PresidentialDebate2020,,,,"{'user_id': None, 'username': None}",,,
1,1318938583122743296,1318938583122743296,2020-10-21 15:33:57,2020-10-21 15:33:57,0,,Be prepared for Trump to railroad Thursday’s d...,en,[presidentialdebate2020],[],...,0,https://twitter.com/donaldjtrumpjr/status/1318...,PresidentialDebate2020,,,,"{'user_id': None, 'username': None}",,,
2,1318932554897031168,1318932554897031168,2020-10-21 15:10:00,2020-10-21 15:10:00,0,,Join us tomorrow from 5-8pm as @michaelpleahy ...,en,[presidentialdebate2020],[],...,0,,PresidentialDebate2020,,,,"{'user_id': None, 'username': None}",,,
3,1318928783169245184,1318928783169245184,2020-10-21 14:55:01,2020-10-21 14:55:01,0,,Wanna bet #ProudBoys comes up #PresidentialDeb...,en,"[proudboys, presidentialdebate2020]",[],...,0,,PresidentialDebate2020,,,,"{'user_id': None, 'username': None}",,,
4,1318927150247018496,1318927150247018496,2020-10-21 14:48:31,2020-10-21 14:48:31,0,,RT College Tour @BelmontUniv was spotless. Gor...,en,"[musiccity, presidentialdebate2020]",[],...,0,,PresidentialDebate2020,,,,"{'user_id': None, 'username': None}",,,


In [ ]:
# write everything into one function that can be re-used later
def text_prepro(texts):
  """
  takes in a pandas series (1 column of a DF)
  removes twitter stuff
  lowercases, normalizes text
  """
  texts_clean = texts.map(lambda t: prepro.clean(t))
  texts_clean = texts_clean.str.replace('#','')

  clean_container = []

  pbar = tqdm.tqdm(total=len(texts_clean),position=0, leave=True)

  for text in nlp.pipe(texts_clean, disable=["tagger", "parser", "ner"]):

    txt = [token.lemma_.lower() for token in text 
          if token.is_alpha 
          and not token.is_stop 
          and not token.is_punct]

    clean_container.append(" ".join(txt))
    pbar.update(1)
  
  return clean_container

In [ ]:
# apply all prepro-pipeline to texts
pol_tweets['text_clean'] = text_prepro(pol_tweets['text'])

  0%|          | 0/50000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
100%|██████████| 50000/50000 [02:04<00:00, 403.03it/s]


In [ ]:
pol_tweets

,text,labels,text_clean
340675,RT @GreenBeretFound Today we remember Sgt. 1st...,0,today remember sgt st class ryan savard killed...
289492,"Yes, yes, yes, yes and yes. 😷 #JerseyStrong 💪🏾...",1,yes yes yes yes yes jerseystrong qt prioritize...
371088,Made new friends this afternoon delivering mas...,1,new friends afternoon delivering masks veteran...
82212,RT @TXMilitary Happening TODAY: Pilots with th...,0,happening today pilots texas air guard conduct...
476047,RT @SteveScalise President Trump's legal team ...,0,president trump legal team completely destroye...
...,...,...,...
61499,Outrageous.\n\nBrave health care workers are p...,0,outrageous brave health care workers putting l...
185562,RT @dskolnick .@RepTimRyan proposes up to $3K ...,1,proposes k month family pandemic
354040,It is clear that the #HeroesAct will help tens...,1,clear heroesact help tens millions americans h...
708686,Democrats are talking about Bolton and Mulvane...,0,democrats talking bolton mulvaney witnesses co...


In [ ]:
# renaming and reordering

pol_tweets_df = pd.DataFrame({'label':pol_tweets['labels'], 'text':pol_tweets['text_clean']})

In [ ]:

pol_tweets_df.label.value_counts().reset_index()

,index,label
0,1,31181
1,0,18819


In [ ]:
alt.Chart(pol_tweets_df.label.value_counts().reset_index()).mark_bar(filled=True).encode(
    alt.X('label:Q', title='N Tweets'),
    alt.Y('index:N', title='Category')
)

alt.Chart(...)

In [ ]:
# fixing sample imbalance
rus = RandomUnderSampler(random_state=42)
data_df_res, y_res = rus.fit_resample(pol_tweets_df, pol_tweets_df['label'])

In [ ]:
data_df_res['label'].value_counts()

0    18819
1    18819
Name: label, dtype: int64

In [ ]:
# Splitting the dataset into the Training set and Test set (since we have a new output variable)
X_train, X_test, y_train, y_test = train_test_split(data_df_res['text'], y_res, test_size = 0.4, random_state = 42)

In [ ]:
#instantiate models and "bundle up as pipeline"

tfidf = TfidfVectorizer()
cls = LogisticRegression()

pipe = make_pipeline(tfidf, cls)

In [ ]:
pipe.fit(X_train,y_train) # fit model

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression())])

In [ ]:
# evaluate model performance on training set

y_eval = pipe.predict(X_train)
report = classification_report(y_train, y_eval)
print(report)

              precision    recall  f1-score   support

           0       0.85      0.87      0.86     11292
           1       0.87      0.84      0.85     11290

    accuracy                           0.86     22582
   macro avg       0.86      0.86      0.86     22582
weighted avg       0.86      0.86      0.86     22582



In [ ]:
# overall weights (works only for linear models)
eli5.show_weights(pipe, top=20, target_names=['Rep','Dem'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#instantiate models and "bundle up as pipeline"

tfidf = TfidfVectorizer()
svd = TruncatedSVD(n_components = 100)
cls_xg = XGBClassifier()

pipe_xg = make_pipeline(tfidf, svd, cls_xg)

In [ ]:
pipe_xg.fit(X_train,y_train) # fit model

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('truncatedsvd', TruncatedSVD(n_components=100)),
                ('xgbclassifier', XGBClassifier())])

In [ ]:
# evaluate model performance on training set

y_eval = pipe_xg.predict(X_train)
report = classification_report(y_train, y_eval)
print(report)

              precision    recall  f1-score   support

           0       0.69      0.74      0.71     11292
           1       0.72      0.67      0.69     11290

    accuracy                           0.70     22582
   macro avg       0.70      0.70      0.70     22582
weighted avg       0.70      0.70      0.70     22582



In [ ]:
# evaluate model performance on test set

y_pred = pipe_xg.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.66      0.71      0.68      7527
           1       0.68      0.63      0.66      7529

    accuracy                           0.67     15056
   macro avg       0.67      0.67      0.67     15056
weighted avg       0.67      0.67      0.67     15056



In [ ]:
debate_tweets.at[0, 'tweet']

'Still time to register: Students can join the @UVADemocracy Student Advisory Council for a socially-distanced Presidential Debate Watch Party on Thursday, Oct. 22 at 9 PM. #PresidentialDebate2020 @noah_strike  Register:  https://t.co/EeVPuiVdtn  https://t.co/dONWhD2B99'

In [ ]:
# run single prediction

t1 = debate_tweets.at[0, 'tweet']

In [ ]:
# preprocess

t1_p = text_prepro(pd.Series(t1)) # note, we need to pack text up as pd.Series 

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
100%|██████████| 1/1 [00:00<00:00, 91.18it/s]


In [ ]:
# predict

pipe.predict(t1_p)

array([1])

In [ ]:
# overall weights (works only for linear models)
eli5.show_weights(pipe, top=10, target_names=['Rep','Dem'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# explain one prediction
eli5.show_prediction(pipe[1], t1_p[0], vec=pipe[0],
                     target_names=['Rep','Dem'])

In [ ]:
# apply all prepro-pipeline to texts
debate_tweets['text_clean'] = text_prepro(debate_tweets['tweet'])

  0%|          | 0/8811 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
100%|██████████| 8811/8811 [00:13<00:00, 644.66it/s]


In [ ]:
debate_tweets['class_pred'] = pipe.predict(debate_tweets['text_clean'])

In [ ]:
pipe.predict_proba(debate_tweets['text_clean'])

array([[0.18435707, 0.81564293],
       [0.55675874, 0.44324126],
       [0.26404614, 0.73595386],
       ...,
       [0.45177733, 0.54822267],
       [0.41030131, 0.58969869],
       [0.70390224, 0.29609776]])

In [ ]:
debate_tweets['rep_proba'] = pipe.predict_proba(debate_tweets['text_clean'])[:,1]

In [ ]:
debate_tweets.sort_values(['rep_proba'], ascending=False)['tweet'][:10]

344     @mjfree @CNN #Trump Don't have to show up!!!!!...
3764    Trump caved!!!! 😂😂😂#JoeBidenKamalaHarris2020 #...
7989    #Trump  Mute him out Mute him out Mute him out...
7467    Fight for Trump  ##Trupm  #PresidentialDebate2...
298     Trump can thank Trump for this one 👇#President...
1396    Whose paying that black women behind trump to ...
5984    Trump put Americans health first??? WHEN??? #P...
1449    The Me Me Me Show by Donald Trump. #Presidenti...
7687    It’s trump getting Covid for me👀 #trump #presi...
7046    #ChrisWallace  #TrumpHasCovid  #TestTraceIsola...
Name: tweet, dtype: object